In [ ]:
import pandas as pd
import numpy as np
import wave
import os
import winsound
import pydub
import time
from pydub import AudioSegment
from pathlib import Path

In [8]:
# C = 0, C# = 1, etc.
roots = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# 1 |-> sus 2, 2 |-> minor 3rd, 3 |-> major 3rd, 4 |-> sus 4
thirds = [1,2,3,4]

# 1 |-> flat 5th, 2 |-> perfect 5th, 3 |-> sharp 5th
fifths = [1,2,3]

# 0|-> no 7th, 1 |-> minor 7th, 2 |-> major 7th
sevenths = [0,1,2]

# 0 |-> no 9th, 1 |-> flat 9th, 2 |-> natural 9th, 3 |-> sharp 9th
ninths = [0, 1, 2, 3]

# 0 |-> no 11th, 1 |-> natural 11th, 2 |-> sharp 11th
elevenths = [0, 1, 2]

# 0 |-> no 13th, 1 |-> flat 13th, 2 |-> natural 13th
thirteenths = [0, 1, 2]

In [16]:
def chord_name(chord, shift=0):
    # Inputs: chord; an array, shift, an integer
    # the chord array has a choice of root note, and the remaining elements
    # are the choice of extensions
    
    # Outputs: a string which is the name of the chord
    
    name = ""
    
    # Next we define a some dictionaries using our chord note conventions:
    root_dict = {
    0: 'C',
    1: 'C#',
    2: 'D',
    3: 'Eb',
    4: 'E',
    5: 'F',
    6: 'F#',
    7: 'G',
    8: 'G#',
    9: 'A',
    10: 'Bb',
    11: 'B'
    }
    
    third_dict = {
        1: 'sus2',
        2: 'm',
        3: '',
        4: 'sus4'
    }
    
    fifth_dict = {
        1: ' b5',
        2: '',
        3: ' #5'
    }
    
    seventh_dict = {
        0: '',
        1: ' 7',
        2: ' maj7'
    }
    
    ninth_dict = {
        0: '',
        1: ' b9',
        2: ' 9',
        3: ' #9'
    }
    
    eleventh_dict = {
        0: '',
        1: ' 11',
        2: ' #11'
    }
    
    thirteenth_dict = {
        0: '',
        1: ' b13',
        2: ' 13'
    }
    
    chord_array_dict = {
        0: root_dict,
        1: third_dict,
        2: fifth_dict,
        3: seventh_dict,
        4: ninth_dict,
        5: eleventh_dict,
        6: thirteenth_dict
    }
    
    for i in range(7):
        if i == 0:
            name += chord_array_dict[i][chord[i]+shift]
        else:
            name += chord_array_dict[i][chord[i]]
        
    return name

There are 5184 (some other number) naive chords, and a naive chord is a 7-tuple in this setup.
Thus our data frame will be a $3456 \times 7$ matrix.

Update: I realized that minor 3rd and sharp 9 are equivalent. After a basic combinatorics exercise, I determined there were 432 chords with a minor 3rd and sharp 9 simulataneously. So the new correct number of chords is 3024.

Update: Yet again, I realized that chords with both flat 5 and sharp 11 are redundant. I will thus remove all chords that have simultaneously a flat 5 and sharp 11. There are 576 of these chords, bringing the total number of chords down to 2448.
However, I realized it's unreasonable to require all chords to have a 7th degree extension, so I have decided to include chords that don't have a 7th extension. This thus brings the total number of chords to $5184-648-864+108=3780$ chords. Here $5184$ is the size of the direct product, $648$ is the number of minor 3rd/sharp 9 redundancies, $864$ is the number of flat 5/sharp 11 redundancies, and 108 is the size of the intersection of these two redundant sets.

In [22]:
chord_data = np.zeros((15552, 7))

def cartesian_coord(*arrays):
    grid = np.meshgrid(*arrays)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    points[:, [1, 0]] = points[:, [0, 1]]
    return points

chord_data = cartesian_coord(
    thirds, 
    roots, 
    fifths, 
    sevenths, 
    ninths, 
    elevenths, 
    thirteenths
)

chord_data=pd.DataFrame(chord_data, columns=['root', '3rd', '5th', '7th', '9th', '11th', '13th'])

#add a chord name column
names = [""]*15552
for index, row in chord_data.iterrows():
    names[index]=chord_name(row)
    
chord_data['name'] = names

#remove redundant chords:

#remove chords with simultaneously minor 3rd and sharp 9
chord_data=chord_data[(chord_data["3rd"]!=2) |(chord_data["9th"]!=3)]
#remove chords with simultaneously flat 5 and sharp 11
chord_data=chord_data[(chord_data["5th"]!=1) |(chord_data["11th"]!=2)]
#remove chords with sharp 5 and b13
chord_data=chord_data[(chord_data["5th"]!=3) | (chord_data["13th"]!=1)]
#remove chords with sharp 5 and sharp 11, equivalent to b5 and b13
chord_data=chord_data[(chord_data["5th"]!=3) | (chord_data["11th"]!=2)]
#remove chords with sus2 and sharp 9, equivalent to a minor 9 chord
chord_data=chord_data[(chord_data['3rd']!=1) | (chord_data['9th']!=3)]
#remove chords with sus2 and natural 9, redundant note
chord_data=chord_data[(chord_data['3rd']!=1) | (chord_data['9th']!=2)]
#remove chords with sus4 and natural 11, redundant note
chord_data=chord_data[(chord_data['3rd']!=4) | (chord_data['11th']!=1)]
#remove chords with sus4 and sharp 9, equivalent to minor 11 chord
chord_data=chord_data[(chord_data['3rd']!=4) | (chord_data['9th']!=3)]
#remove chords with sus4 and natural 9, equivalent to sus2 11
chord_data=chord_data[(chord_data['3rd']!=4) | (chord_data['9th']!=2)]

chord_data.to_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    header='infer', 
    index=None, 
    sep=' ', 
    mode='a'
)

In [37]:
def keys_of_chord(chord):
    chord = np.array(chord)
    offset = np.array([0, chord[0]+2, chord[0]+5, chord[0]+9, chord[0]+0, chord[0]+4, chord[0]+7])
    keys = (chord+offset)%12
    del_inds = [i for i in range(3,7) if chord[i] == 0]
    keys=np.delete(keys, del_inds)
    return keys

In [23]:
chords = pd.read_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    sep=' '
)
chords

,root,3rd,5th,7th,9th,11th,13th,name
0,0,1,1,0,0,0,0,Csus2 b5
1,0,1,1,0,0,0,1,Csus2 b5 b13
2,0,1,1,0,0,0,2,Csus2 b5 13
3,0,1,1,0,0,1,0,Csus2 b5 11
4,0,1,1,0,0,1,1,Csus2 b5 11 b13
...,...,...,...,...,...,...,...,...
6943,11,4,3,1,1,0,2,Bsus4 #5 7 b9 13
6944,11,4,3,2,0,0,0,Bsus4 #5 maj7
6945,11,4,3,2,0,0,2,Bsus4 #5 maj7 13
6946,11,4,3,2,1,0,0,Bsus4 #5 maj7 b9


In [ ]:
for i, chord1 in chords.iterrows():
    chord1 = chord1[:-1]
    print(i)
    for j, chord2 in chords.iterrows():
        chord2 = chord2[:-1]
        if j > i:
            if (set(keys_of_chord(chord1)) == set(keys_of_chord(chord2))):
                print("Chord 1: ")
                print(chords.iloc[i].values)
                print("Chord 2: ")
                print(chords.iloc[j].values)
                print("Notes 1: ", set(keys_of_chord(chord1)))
                print("Notes 2: ", set(keys_of_chord(chord2)))

In [ ]:
chord_data.to_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    header='infer', 
    index=None, 
    sep=' ', 
    mode='a'
)

In [55]:
print(set(keys_of_chord([9, 1, 3, 0, 0, 0, 0])))
print(keys_of_chord([5, 2, 2, 0, 0, 0, 0]).sort)

{0, 9, 5}
<built-in method sort of numpy.ndarray object at 0x0000012C0CFE5A30>
